In [4]:
import pandas as pd


# Part 1


Considering only the first bit of each number, there are five 0 bits and seven 1 bits. Since the most common bit is 1, the first bit of the gamma rate is 1.

The most common second bit of the numbers in the diagnostic report is 0, so the second bit of the gamma rate is 0.

The most common value of the third, fourth, and fifth bits are 1, 1, and 0, respectively, and so the final three bits of the gamma rate are 110.


## Test


In [5]:
test = pd.read_csv("test", header=None, dtype=str, names=["raw"])
test = pd.DataFrame(test.apply(lambda x: list(*x), axis=1).tolist())
test = test.astype("int8")
test


,0,1,2,3,4
0,0,0,1,0,0
1,1,1,1,1,0
2,1,0,1,1,0
3,1,0,1,1,1
4,1,0,1,0,1
5,0,1,1,1,1
6,0,0,1,1,1
7,1,1,1,0,0
8,1,0,0,0,0
9,1,1,0,0,1


becaus the epsilon rate is the inverse I thought it best to be proactive


In [6]:
test_inv = ~test + 2
test_inv


,0,1,2,3,4
0,1,1,0,1,1
1,0,0,0,0,1
2,0,1,0,0,1
3,0,1,0,0,0
4,0,1,0,1,0
5,1,0,0,0,0
6,1,1,0,0,0
7,0,0,0,1,1
8,0,1,1,1,1
9,0,0,1,1,0


So, the gamma rate is the binary number 10110, or 22 in decimal.

The epsilon rate is calculated in a similar way; rather than use the most common bit, the least common bit from each position is used. So, the epsilon rate is 01001, or 9 in decimal. Multiplying the gamma rate (22) by the epsilon rate (9) produces the power consumption, 198.


In [7]:
test_out = pd.DataFrame({0: test_inv.sum(), 1: test.sum()})
test_out["gamma"] = test_out.apply(lambda x: 1 if x[1] > x[0] else 0, axis=1)
test_out["epsilon"] = test_out.apply(lambda x: 1 if x[1] < x[0] else 0, axis=1)
test_out


,0,1,gamma,epsilon
0,5,7,1,0
1,7,5,0,1
2,4,8,1,0
3,5,7,1,0
4,7,5,0,1


In [8]:
vals = "".join([str(b) for b in test_out["gamma"].tolist()]), "".join(
    [str(b) for b in test_out["epsilon"].tolist()]
)
ints = [int(b, 2) for b in vals]
ints[0] * ints[1]


198

## Actuall run


In [9]:
df = pd.read_csv("input", header=None, dtype=str, names=["raw"])
df = pd.DataFrame(df.apply(lambda x: list(*x), axis=1).tolist())
df = df.astype("int8")
df_inv = ~df + 2
df


,0,1,2,3,4,5,6,7,8,9,10,11
0,1,1,0,1,1,0,0,0,0,1,0,0
1,0,1,0,1,1,0,0,1,1,1,0,0
2,0,0,1,0,0,1,1,0,1,0,1,0
3,0,1,1,0,1,1,1,0,1,0,1,0
4,1,0,1,1,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
995,0,0,1,1,1,1,0,1,1,1,0,0
996,1,1,1,1,1,0,0,1,1,0,0,1
997,0,1,0,0,1,0,1,1,0,0,1,1
998,1,1,0,0,1,1,0,1,1,0,0,1


In [10]:
df_out = pd.DataFrame({0: df_inv.sum(), 1: df.sum()})
df_out["gamma"] = df_out.apply(lambda x: 1 if x[1] > x[0] else 0, axis=1)
df_out["epsilon"] = df_out.apply(lambda x: 1 if x[1] < x[0] else 0, axis=1)
df_out


,0,1,gamma,epsilon
0,514,486,0,1
1,502,498,0,1
2,494,506,1,0
3,493,507,1,0
4,509,491,0,1
5,505,495,0,1
6,475,525,1,0
7,515,485,0,1
8,513,487,0,1
9,502,498,0,1


In [11]:
vals = "".join([str(b) for b in df_out["gamma"].tolist()]), "".join(
    [str(b) for b in df_out["epsilon"].tolist()]
)
ints = [int(b, 2) for b in vals]
print(f"{vals=:}")
print(f"Hex= {hex(ints[0]), hex(ints[1])}")
print(f"Answer= {ints[0] * ints[1]}")


vals=('001100100010', '110011011101')
Hex= ('0x322', '0xcdd')
Answer= 2640986


# Part 2


In [14]:
def getCounts(series):
    return {0: len(series.loc[series == 0]), 1: len(series.loc[series == 1])}


def getLagest(series):
    counts = getCounts(series)
    return 0 if counts[0] > counts[1] else 1


def getSmallest(series):
    counts = getCounts(series)
    return 0 if counts[0] <= counts[1] else 1

def getOxGenStatus(dataframe: pd.DataFrame, filter_function) -> int:
    tmp_df = dataframe
    for col in dataframe:
        filter_val = filter_function(tmp_df[col])
        tmp_df = tmp_df.loc[tmp_df[col] == filter_val]
        if len(tmp_df) == 1:
            break
    return int(tmp_df.transpose().astype(str).sum().to_list()[0], 2)



### Testing part 2


In [26]:
ox, co = (getOxGenStatus(test, getLagest), getOxGenStatus(test, getSmallest))
print(f"{ox = } * {co = } = {ox * co}")

ox = 23 * co = 10 = 230


### Running part 2


In [27]:
ox, co = (getOxGenStatus(df, getLagest), getOxGenStatus(df, getSmallest))
print(f"{ox = } * {co = } = {ox * co}")

ox = 1883 * co = 3623 = 6822109
